In [1]:
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
from torch.optim import Adam
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os
from open_clip import create_model_and_transforms
from PIL import Image

In [2]:
class ImageCaptioningModel(nn.Module):
    def __init__(self, encoder_model_name='hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224', 
                 gpt2_model_name='gpt2', projection_dim=512):
        super(ImageCaptioningModel, self).__init__()
        
        # Load Encoder model and its tokenizer
        self.encoder_model, self.preprocess_train, self.preprocess_val = create_model_and_transforms(
            encoder_model_name, pretrained_hf=True
        )
        
        # Get the encoder output size by passing a dummy input through the encoder
        dummy_input = torch.rand(1, 3, 224, 224)  # Assuming the model input size is 224x224
        with torch.no_grad():
            dummy_output = self.encoder_model.encode_image(dummy_input)
        self.encoder_output_dim = dummy_output.shape[1]  # Use the embedding dimension from the dummy output
        
        # GPT-2 Decoder model
        self.gpt2_model = GPT2LMHeadModel.from_pretrained(gpt2_model_name)
        self.gpt2_tokenizer = GPT2Tokenizer.from_pretrained(gpt2_model_name)

        # Projection layer to match embedding sizes
        self.image_embeddings_projected = nn.Linear(self.encoder_output_dim, projection_dim)
    
    def forward(self, images, input_ids, attention_mask):
        # Extract image embeddings
        image_embeddings = self.encoder_model.encode_image(images)
        image_embeddings_projected = self.image_embeddings_projected(image_embeddings)
        # Get GPT-2 outputs using the provided input_ids and attention_mask
        logits = self.gpt2_model(input_ids=input_ids, attention_mask=attention_mask).logits
        return logits
 


    def preprocess_image(self, image_path):
        image = Image.open(image_path).convert('RGB')
        image_tensor = self.preprocess_val(image).unsqueeze(0)  # Preprocess image
        return image_tensor

    def tokenize_caption(self, caption):
        return self.gpt2_tokenizer.encode(caption, return_tensors='pt')

    def decode_caption(self, caption_tokens):
        return self.gpt2_tokenizer.decode(caption_tokens, skip_special_tokens=True)


In [3]:
class ROCODataset(Dataset):
    def __init__(self, caption_file, image_folder, model):
        self.captions = pd.read_csv(caption_file)
        self.image_folder = image_folder
        self.preprocess = model.preprocess_train
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

        # Assign a padding token
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token  # or you can set a new token with add_special_tokens

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        caption = self.captions.iloc[idx]['Caption']
        image_id = self.captions.iloc[idx]['ID']
        image_path = os.path.join(self.image_folder, image_id + '.jpg')

        # Load and preprocess the image
        image = Image.open(image_path).convert("RGB")
        image_tensor = self.preprocess(image)
        # Tokenize the caption
        caption_tokens = self.tokenizer(caption, return_tensors='pt', padding=True, truncation=True)

        return image_tensor, caption_tokens['input_ids'].squeeze(0), caption_tokens['attention_mask'].squeeze(0)


In [4]:
def custom_collate_fn(batch):
    images, captions, attention_masks = zip(*batch)
    
    # Stack images (they are all the same size)
    images = torch.stack(images)
    
    # Pad captions and attention masks
    captions = torch.nn.utils.rnn.pad_sequence(captions, batch_first=True, padding_value=0)
    attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True, padding_value=0)
    
    return images, captions, attention_masks


In [5]:
import time

def train_model_mixed_precision(model, dataloader, epochs=5, lr=1e-4):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = Adam(model.parameters(), lr=lr)
    scaler = torch.amp.GradScaler()
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        running_loss = 0.0
        epoch_start_time = time.time()

        for i, (image_tensor, inputs, attention_mask) in enumerate(dataloader):
            image_tensor = image_tensor.to(device)
            inputs = inputs.to(device)
            attention_mask = attention_mask.to(device)

            optimizer.zero_grad()

            with torch.amp.autocast(device_type='cuda'):
                logits = model(image_tensor, inputs, attention_mask)
                loss = loss_fn(logits.view(-1, logits.size(-1)), inputs.view(-1))

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

            # Measure time per batch
            if i % 10 == 0:  # Print every 10 batches
                print(f"Batch {i}, Loss: {loss.item()}")

        epoch_duration = time.time() - epoch_start_time
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader)}, Duration: {epoch_duration:.2f} seconds")


In [6]:
# File paths
train_caption_file = "../Datasets/ROCO2/train_captions.csv"
train_image_folder = "../Datasets/ROCO2/train_images/train/"
test_caption_file = "../Datasets/ROCO2/test_captions.csv"
test_image_folder = "../Datasets/ROCO2/test_images/test/"

# Initialize the model (use the BiomedCLIP model as encoder)
model = ImageCaptioningModel(encoder_model_name='hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

# Create datasets for train and test
train_dataset = ROCODataset(train_caption_file, train_image_folder, model)
test_dataset = ROCODataset(test_caption_file, test_image_folder, model)

# Create DataLoaders for train and test
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, collate_fn=custom_collate_fn)

G:\anaconda3\envs\MEDCAT\Lib\site-packages\open_clip\factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=map_

In [ ]:
# Train the model with the ROCO dataset
train_model_mixed_precision(model, train_loader, epochs=5)


G:\anaconda3\envs\MEDCAT\Lib\site-packages\timm\models\vision_transformer.py:92: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  x = F.scaled_dot_product_attention(


Batch 0, Loss: 9.890169143676758
Batch 10, Loss: 4.129048824310303
Batch 20, Loss: 1.3760032653808594
Batch 30, Loss: 0.2009284943342209
Batch 40, Loss: 0.036839503794908524
Batch 50, Loss: 0.03655512258410454
Batch 60, Loss: 0.011183960363268852
Batch 70, Loss: 0.015059743076562881
Batch 80, Loss: 0.007550396490842104
Batch 90, Loss: 0.004434171598404646
